In [1]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Activation
from tensorflow.keras.datasets import mnist
from nn2graph import dense_model_to_graph
import graph_tool as gt
import numpy as np

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
hidden_dims = 32
input_dims = X_train.shape[1]*X_train.shape[2]
input_0 = Input((input_dims,))
relu = Activation('relu')
softmax = Activation('softmax')
dense_1 = Dense(hidden_dims, kernel_initializer='he_uniform')
dense_2 = Dense(hidden_dims, kernel_initializer='he_uniform')
dense_3 = Dense(hidden_dims, kernel_initializer='he_uniform')
dense_out = Dense(10, kernel_initializer='he_uniform')
output_0 = softmax(dense_out(dense_3(relu(dense_2(relu(dense_1(input_0)))))))
model = Model(inputs=input_0, outputs=output_0)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [4]:
layers = model.layers
for layer in layers:
    print(layer)
dense_layers = [l for l in layers if isinstance(l, Dense)]

In [5]:
g = dense_model_to_graph(model)
print(g)

NameError: name 'Dense' is not defined